# Applying Machine Learning and Deep Learning to identify home appliances consuming excess power

## Copyright (c) 2018, Faststream Technologies
## Author: Sudhanva Narayana

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the dataset
dataset = pd.read_csv('../data/home_final.csv')
X = dataset.iloc[:, [0, 3, 4, 5, 6, 7, 9, 10]].values
y = dataset.iloc[:, 11].values

### Data Preprocessing

In [3]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelencoder_X_0 = LabelEncoder()
X[:, 0] = labelencoder_X_0.fit_transform(X[:, 0])
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
labelencoder_X_4 = LabelEncoder()
X[:, 4] = labelencoder_X_4.fit_transform(X[:, 4])
labelencoder_X_5 = LabelEncoder()
X[:, 5] = labelencoder_X_5.fit_transform(X[:, 5])

In [4]:
onehotencoder = OneHotEncoder(categorical_features=[0, 1, 2, 4, 5])
hot_X = onehotencoder.fit_transform(X).toarray()

In [5]:
columns = dataset.columns
dummies = []
dummies_sum = 0
categories = [0, 1, 2, 4, 5]

for category in categories:
    dummies_sum += category * (dataset.iloc[:, category].unique().size)
    dummies.append(dummies_sum)

In [6]:
# Removing dummy variables
hot_X = np.delete(hot_X, dummies, 1)

C:\Users\Sudhanva\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  


In [7]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(hot_X, y, test_size=0.2, random_state=0)

In [8]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

### Building the Artificial Neural Network

In [9]:
# import keras library and packages
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

# Initializing the ANN
classifier = Sequential()

Using TensorFlow backend.


In [10]:
# Adding the input layer and the first hidden layer
layer_info = Dense(activation='relu', input_dim=134, units=65, kernel_initializer='uniform')
classifier.add(layer_info)

# Adding second hidden layer
layer_info = Dense(activation='relu', units=65, kernel_initializer='uniform')
classifier.add(layer_info)

# Adding third hidden layer
layer_info = Dense(activation='relu', units=65, kernel_initializer='uniform')
classifier.add(layer_info)

# Adding output layer
layer_info = Dense(activation='sigmoid', units=1, kernel_initializer='uniform')
classifier.add(layer_info)

In [11]:
# Compiling the ANN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
# Fitting the ANN to the training set
with tf.device('/GPU:0'):
    classifier.fit(X_train, y_train, batch_size=100, epochs=100)   

Epoch 1/100
4166/4166 [==============================] - 2s 567us/step - loss: 0.6931 - acc: 0.5024
Epoch 2/100
4166/4166 [==============================] - 0s 36us/step - loss: 0.6860 - acc: 0.5723
Epoch 3/100
4166/4166 [==============================] - 0s 34us/step - loss: 0.6287 - acc: 0.6426
Epoch 4/100
4166/4166 [==============================] - 0s 33us/step - loss: 0.5571 - acc: 0.7026
Epoch 5/100
4166/4166 [==============================] - 0s 33us/step - loss: 0.4595 - acc: 0.7734
Epoch 6/100
4166/4166 [==============================] - 0s 33us/step - loss: 0.3660 - acc: 0.8289
Epoch 7/100
4166/4166 [==============================] - 0s 33us/step - loss: 0.3123 - acc: 0.8579
Epoch 8/100
4166/4166 [==============================] - 0s 33us/step - loss: 0.2623 - acc: 0.8850
Epoch 9/100
4166/4166 [==============================] - 0s 33us/step - loss: 0.2282 - acc: 0.8975
Epoch 10/100
4166/4166 [==============================] - 0s 33us/step - loss: 0.2029 - acc: 0.9131
Epoch 11

In [13]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [14]:
cm

array([[407, 109],
       [157, 369]], dtype=int64)